In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt

# dataset is abot crack in concrete or not
it is a classification task.

In [2]:
import os
import cv2

In [3]:
positive_dir="C:/Users/Satyam/Downloads/Positive"

In [4]:
negative_dir="C:/Users/Satyam/Downloads/Negative"

In [5]:
def load_images(directory):
    images=[]
    for filename in os.listdir(directory):
        img_path=os.path.join(directory,filename)
        img=cv2.imread(img_path)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        images.append(img)
    return images

In [ ]:
positive_images=load_images(positive_dir)
negative_images=load_images(negative_dir)

In [ ]:
positive_images_train=positive_images[6000:7000]
negative_images_train=negative_images[6000:7000]

# here we scaled data between [0,1]
scalling is required because here we use ResNet50, and REsnet is pre traineed on scalled data
scalling is also rquired because it heps model to prevnt overfitting
scallin converts the pixels in between[0-1] it makes easier to convergence

In [ ]:
imgs=[]
def make_norm(abcd):
    for image in abcd:
        image=image.astype(np.float32)
        normalized_image=image/255.0
        imgs.append(normalized_image)
    return imgs
    
    

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
positive_images_train=np.array(make_norm(positive_images_train))

In [ ]:
positive_images_train.shape

In [ ]:
negative_images_train.shape

In [ ]:
negative_images_train=np.array((negative_images_train))

In [ ]:
x=np.concatenate([positive_images_train,negative_images_train],axis=0)

In [ ]:
y=(np.concatenate([np.ones(len(positive_images_train)),np.zeros(len(negative_images_train))],axis=0))


In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
x.shape

In [ ]:
y.shape

We imported ResNet50  a pretrained model for this classification task, this model is family of ResNet model, and it have 50 layers,
so it is callled ResNet50

In [ ]:
resnet_model=tf.keras.applications.ResNet50(weights='imagenet',include_top=False,input_shape=(227,227,3))

In [ ]:
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
a=GlobalAveragePooling2D()(resnet_model.output)
a=Dense(128,activation='relu')(a)
predictions=Dense(1,activation='sigmoid')(a)

In [ ]:
model=Model(inputs=resnet_model.input,outputs=predictions)
for layers in resnet_model.layers:
    layers.trainable=False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
pos_val=np.array(positive_images[1000:2000])
neg_val=np.array(negative_images[1000:2000])

In [ ]:
x_val=np.concatenate([pos_val,neg_val],axis=0)
y_val=y=np.concatenate([np.array(np.ones(len(pos_val))),np.array(np.zeros(len(neg_val)))],axis=0)

In [ ]:
model.fit(x,y,epochs=1,batch_size=62,validation_data=(x_val,y_val))

In [ ]:
def balanced_batch_generator(a,b,batch_size=32):
    num_classes=len(np.unique(b))
    class_indices=[np.where(b==i)[0] for i in range(num_classes)]
    class_count=[len(indices) for indices in class_indices]
    while True:
        batch_indices=[]
        for i in range(batch_size//2):
            class_index=np.random.randint(num_classes)
            indices=class_indices[class_index]
            index=np.random.choice(indices)
            batch_indices.append(index)
        yield a[batch_indices],b[batch_indices]
        
        

In [ ]:
batch_size=62
train_generator=balanced_batch_generator(x,y,batch_size)

In [ ]:
steps_per_epochs=len(x)//batch_size

In [ ]:
from tensorflow.keras import callbacks
early_stopping=callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

it is always good to experiment with No of epochs,early_stopping,

In [ ]:
history=model.fit(train_generator,
                 steps_per_epoch=steps_per_epochs,
                 epochs=3,
                 validation_data=((x_val,y_val)),
                 callbacks=[early_stopping]
                 )

In [ ]:
train_loss=history.history['loss']
val_loss=history.history['val_loss']
train_acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

In [ ]:
plt.plot(train_loss,label='trainig_loss')
plt.plot(val_loss,label='validation_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')

In [ ]:
plt.plot(train_acc,label='trainning Accuracy')
plt.plot(val_acc,label='validation_loss')
plt.xlabel('epochs')
plt.ylabel('Accuracy')

In [ ]:
positive_test=np.array(positive_images[3000:3050])
negative_test=np.array(negative_images[7850:7900])
positive_pred=model.predict(positive_test)

threshold=.45
pos_predicted_label=(positive_pred>threshold).astype(int)


In [ ]:
from sklearn.metrics import precision_score,recall_score
precision=precision_score(np.ones(50),pos_predicted_label)

In [ ]:
precision

In [ ]:
recall=recall_score(np.ones(50),pos_predicted_label)

In [ ]:
recall